# Installation

In [ ]:
# Mount gdrive for data sets
import os
from google.colab import drive
drive.mount('/gdrive')
try:
  os.symlink('/gdrive/My Drive/PHD/Tez', '/drive')
except FileExistsError:
  print("Seems like already mounted the gdrive, skipping the mounting part.")

# Download source code and copy to mounted drive
!rm -f master.zip
!wget https://github.com/aligokalppeker/hypelcnn/archive/refs/heads/master.zip
!unzip -d /drive/ -o -q master.zip && rm master.zip
!rm -rf /drive/hypelcnn/ && mv /drive/hypelcnn-master/ /drive/hypelcnn/

# Add source root to python PATH
import os
os.environ["PYTHONPATH"] += ":/drive/hypelcnn"
!echo $PYTHONPATH

# Python package setup
%tensorflow_version 1.x
!apt-get --assume-yes install libcupti-dev
!pip install colour-science
!pip install numpy==1.18.5

# CNN Training and Visualization

In [ ]:
!python -B /drive/hypelcnn/deep_classification_multigpu.py --path=/drive/data --loader_name=GULFPORTALTDataLoader --importer=GeneratorImporter --step=90000 --max_evals=1 --split_count=1 --test_ratio=0 --neighborhood=3 --model_name=CONCNNModelv1 --algorithm_param_path=/drive/hypelcnn/modelconfigs/algorithm_param_output_concnnv1.json --perform_validation --save_checkpoint_steps=10000 --validation_steps=10000 --train_ratio=125 --augmentation_random_threshold=0.50 --augment_data_with_shadow=simple

In [ ]:
!python -B /drive/hypelcnn/load_checkpoint_calc_accuracy.py --path=/drive/data --loader_name=GULFPORTALTDataLoader --importer=GeneratorImporter --model_name=CONCNNModelv1 --neighborhood=3 --batch_size=4000 --base_log_path=/drive/hypelcnn/log/episode_0/run_0/model.ckpt-0 --output_path=/drive/hypelcnn/log/ --algorithm_param_path=/drive/hypelcnn/modelconfigs/algorithm_param_output_concnnv1.json

# Classic ML Tests

In [ ]:
!python -B /drive/hypelcnn/classic_ml_trainer.py --path=/drive/data --loader_name=GULFPORTDataLoader --neighborhood=1 --batch_size=20000 --split_count=1

# Learn Shadow via GAN

In [ ]:
!python -B /drive/hypelcnn/gan/gan_train_for_shadow.py --path=/drive/data --use_target_map=False --max_number_of_steps=800000 --loader_name=AVONDataLoader --batch_size=32 --validation_itr_count=1000 --use_identity_loss=False --cycle_consistency_loss_weight=10 --gan_type=cycle_gan --regularization_support_rate=0.0

In [ ]:
!python -B /drive/hypelcnn/gan/gan_infer_for_shadow.py --path=/drive/data --checkpoint_path=/drive/hypelcnn/gan/log/model.ckpt-3000 --loader_name=AVONDataLoader --neighborhood=0 --gan_type=cycle_gan

In [ ]:
!python -B /drive/hypelcnn/gan/gan_infer_image_for_shadow.py --path=/drive/data --checkpoint_path=/drive/hypelcnn/gan/log/model.ckpt-3000 --loader_name=AVONDataLoader --make_them_shadow=shadow --output_path=/drive/hypelcnn/gan/log --convert_all=False --gan_type=cycle_gan

In [ ]:
from google.colab import files
files.download('/drive/hypelcnn/gan/log/shadow_image_rgb_shadow_.tif')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# SR CycleGAN Training

In [ ]:
!python -B /drive/hypelcnn/gan/cycle_gann_sr_train.py --path=/drive/data --batch_size=100 --max_number_of_steps=5000

In [ ]:
!python -B /drive/hypelcnn/gan/sr_gann_inference.py --path=/drive/data --checkpoint_path=/drive/hypelcnn/gan/log/model.ckpt-8375 --output_path=/drive/hypelcnn/gan/log